ETL

In [926]:
#Importamos las librerias a usar
import pandas as pd
import re 
import numpy as np

Hoja de hechos

In [927]:
#importamos las hojas del Dataset que se usaran 
df_hechos = pd.read_excel('../Datasets/homicidios.xlsx', sheet_name='HECHOS')

In [928]:
df_hechos.info()#obtenemos una descripción concisa y resumida del DataFrame. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [929]:
df_hechos.head(5)#Visualizamos las 5 primeras filas

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Se evidencia que algunas filas de las columnas victima acusado estan sin datos

In [930]:
df_hechos.groupby(df_hechos['VICTIMA']=='SD')['ID'].count()

VICTIMA
False    687
True       9
Name: ID, dtype: int64

In [931]:
df_hechos.groupby(df_hechos['ACUSADO']=='SD')['ID'].count()

ACUSADO
False    673
True      23
Name: ID, dtype: int64

In [932]:
df_hechos[(df_hechos['VICTIMA']=='SD')&(df_hechos['ACUSADO']=='SD')]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA,...,NaN,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,SD,SD
76,2016-0101,1,2016-08-07,2016,8,7,19:42:00,19,AV GRAL PAZ Y JORGE CHAVEZ,GRAL PAZ,...,NaN,"CHAVEZ, JORGE","PAZ, GRAL. AV. y CHAVEZ, JORGE",9,Point (93946.81673714 97681.68907836),-58.52932872,-34.65014993,SD-SD,SD,SD
92,2016-0119,1,2016-09-04,2016,9,4,06:40:00,6,CRISOLOGO LARRALDE Y AVDA TRIUNVIRATO,AVENIDA,...,NaN,TRIUNVIRATO AV.,"LARRALDE, CRISOLOGO AV. y TRIUNVIRATO AV.",12,Point (96944.69676081 107291.52292157),-58.49659234,-34.56353635,SD-SD,SD,SD
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,...,NaN,NaN,AUTOPISTA 25 DE MAYO,0,Point (. .),.,.,SD-SD,SD,SD
201,2017-0074,1,2017-06-04,2017,6,4,12:50:00,12,CASCO 133,CALLE,...,133.0,NaN,CASCO 133,9,Point (94281.31244858 99043.29027489),-58.52567073,-34.63787791,SD-SD,SD,SD


Se identifica que el dato de la comuna cero parese un error ya que sus datos no son releventes se verifica mas a fondo

In [933]:
df_hechos[df_hechos['COMUNA']==0]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,...,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,...,NaN,NaN,AUTOPISTA 25 DE MAYO,0,Point (. .),.,.,SD-SD,SD,SD


Dado que solo hay dos filas correspondientes a la comuna 0, podemos emplear métodos estadísticos para inferir la ubicación de los incidentes. Sin embargo, la fila 139 será eliminada debido a que solo contiene información sobre una víctima y su fecha, sin otros detalles relevantes. Dado el reducido número de filas con datos faltantes, podemos realizar un análisis estadístico para reemplazar estos valores. Calcularemos la moda por comuna para sustituir los datos faltantes en aquellas filas donde ninguna de las dos columnas tenga información, utilizando el valor de la moda como sustituto.

In [934]:
df_hechos = df_hechos.drop(139, inplace=False)#Elimina las filas 119 

In [935]:
df_hechos['COMUNA'] = df_hechos['COMUNA'].replace(0, df_hechos['COMUNA'].mode()[0])#Le asignamos la moda de las comunas a la comuna 0

In [936]:
# Calcular las modas por comuna 
moda_por_comuna_df = df_hechos.groupby('COMUNA')[['VICTIMA', 'ACUSADO']].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)

In [937]:
# Reemplazar los valores 'SD' en 'ACUSADO' por las modas correspondientes por comuna 
df_hechos['ACUSADO'] = df_hechos.apply(lambda row: moda_por_comuna_df.loc[row['COMUNA'], 'ACUSADO'] if row['ACUSADO'] == 'SD' and row['VICTIMA'] == 'SD'else row['ACUSADO'], axis=1)

Ahora encontramos la moda para cada valor único de cada columna. El objetivo es determinar cuál es la moda de la columna de víctimas cuando se presenta un acusado y viceversa.

In [938]:
df_h_filtrado = df_hechos[(df_hechos['VICTIMA']!= 'SD') & (df_hechos['ACUSADO']!= 'SD')]#Crea un nuevo df para no alterar el original

In [939]:
#Calcula las modas para cada columna
moda_victima_por_acusado = df_h_filtrado.groupby('ACUSADO')['VICTIMA'].agg(lambda x: x.mode().iloc[0] if x.mode().any() else None).to_dict()
moda_acusado_por_victima = df_h_filtrado.groupby('VICTIMA')['ACUSADO'].agg(lambda x: x.mode().iloc[0] if x.mode().any() else None).to_dict()

In [940]:
# Reemplaza los valores 'SD' en 'VICTIMA' por las modas correspondientes
df_hechos['VICTIMA'] = df_hechos['VICTIMA'].replace('SD', np.nan)
df_hechos['VICTIMA'] = df_hechos['VICTIMA'].fillna(df_hechos['ACUSADO'].map(moda_victima_por_acusado))

In [941]:
# Reemplaza los valores 'SD' en 'ACUSADO' por las modas correspondientes
df_hechos['ACUSADO'] = df_hechos['ACUSADO'].replace('SD', np.nan)
df_hechos['ACUSADO'] = df_hechos['ACUSADO'].fillna(df_hechos['VICTIMA'].map(moda_acusado_por_victima))

In [942]:
# Reemplazar los valores 'SD' en 'VICTIMA' por las modas correspondientes por comuna
df_hechos['VICTIMA'] = df_hechos.apply(lambda row: moda_por_comuna_df.loc[row['COMUNA'], 'VICTIMA'] if row['VICTIMA'] == 'SD' else row['VICTIMA'], axis=1)

# Reemplazar los valores 'SD' en 'ACUSADO' por las modas correspondientes por comuna
df_hechos['ACUSADO'] = df_hechos.apply(lambda row: moda_por_comuna_df.loc[row['COMUNA'], 'ACUSADO'] if row['ACUSADO'] == 'SD' else row['ACUSADO'], axis=1)


para finalizar con estas tres columnas se actualiza la columna PARTICIPANTES

In [943]:
df_hechos['PARTICIPANTES'] = df_hechos['VICTIMA'] + '-' + df_hechos['ACUSADO']

Se evidencia que ciertas columnas no tienen el tipo de dato correspontiente

In [944]:
df_hechos[df_hechos['HORA'].str.contains(r'[a-zA-Z]', na=False)]#Verifica si alguna fila de la columna HORA contiene letras.

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
518,2019-0103,1,2019-12-18,2019,12,18,SD,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,...,NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


In [945]:
df_hechos['HORA'].mode().values#muestra cual es la moda

array([datetime.time(14, 0), datetime.time(16, 0)], dtype=object)

En este caso, se puede seleccionar cualquiera de las dos opciones de la moda para reemplazar los valores 'SD' en la columna 'HORA'. Ambas son modas y representan los valores más frecuentes en esta columna, por lo que ambas podrían considerarse representativas del conjunto de datos.

In [946]:
df_hechos.loc[df_hechos['HORA'] == 'SD', 'HORA'] = df_hechos['HORA'].mode().values[0]#Cambia el dato SD por la primera moda

In [947]:
# Se convierte la columna a Datatime
df_hechos['HORA'] = pd.to_datetime(df_hechos['HORA'], format='%H:%M:%S', errors='coerce').dt.time

In [948]:
df_hechos.loc[518]#verificamos la fila 518 donde se realizo el cambio

ID                                                    2019-0103
N_VICTIMAS                                                    1
FECHA                                       2019-12-18 00:00:00
AAAA                                                       2019
MM                                                           12
DD                                                           18
HORA                                                   14:00:00
HH                                                           SD
LUGAR_DEL_HECHO                         PAZ, GRAL. AV. Y GRIVEO
TIPO_DE_CALLE                                          GRAL PAZ
Calle                                            PAZ, GRAL. AV.
Altura                                                      NaN
Cruce                                                    GRIVEO
Dirección Normalizada                   PAZ, GRAL. AV. y GRIVEO
COMUNA                                                       11
XY (CABA)                Point (94643.11

Ahora hacemos la misma verificacion en la columna HH

In [949]:
df_hechos[df_hechos['HH'].str.contains(r'[a-zA-Z]', na=False)]#Verifica si alguna fila contiene letras.

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
518,2019-0103,1,2019-12-18,2019,12,18,14:00:00,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,...,NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


ya que es la misma fila aplicamos el mismo procedimiento solo que esta vez tomaremos la hora de la columna HORA

In [950]:
df_hechos.loc[518, 'HH'] = df_hechos.loc[518, 'HORA'].hour#Asigna la hora de la columna HORA a la coolumna  HH

In [951]:
print(df_hechos.loc[518])#verificamos la fila 518 donde se realizo el cambio

ID                                                    2019-0103
N_VICTIMAS                                                    1
FECHA                                       2019-12-18 00:00:00
AAAA                                                       2019
MM                                                           12
DD                                                           18
HORA                                                   14:00:00
HH                                                           14
LUGAR_DEL_HECHO                         PAZ, GRAL. AV. Y GRIVEO
TIPO_DE_CALLE                                          GRAL PAZ
Calle                                            PAZ, GRAL. AV.
Altura                                                      NaN
Cruce                                                    GRIVEO
Dirección Normalizada                   PAZ, GRAL. AV. y GRIVEO
COMUNA                                                       11
XY (CABA)                Point (94643.11

In [952]:
df_hechos['HH'] = df_hechos['HH'].astype(int)#cambia el tipo de dato de la columna HH a entero

Verificamos las columnas pos x y pos y debido a que su tipo de dato debe ser int y no str

In [953]:
df_hechos['pos x'].value_counts()

pos x
.               11
-58.44451316     5
-58.50073810     4
-58.46743471     4
-58.50877521     4
                ..
-58.39100580     1
-58.37697717     1
-58.45807401     1
-58.42023854     1
-58.37976155     1
Name: count, Length: 605, dtype: int64

In [954]:
df_hechos['pos y'].value_counts()

pos y
.               11
-34.68475866     5
-34.54979510     4
-34.53476874     4
-34.66977709     4
                ..
-34.58498660     1
-34.59573848     1
-34.63997430     1
-34.64234110     1
-34.58679619     1
Name: count, Length: 605, dtype: int64

Se evidencia que en las dos columnas hay 11 filas que contienen '.', verificamos si son las mismas

In [955]:
df_hechos[(df_hechos['pos x']=='.') & (df_hechos['pos y']=='.')]#Filtra de df por las filas coon . en las columnas poos x y pos y

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,...,NaN,NaN,NaN,13,Point (. .),.,.,MOTO-AUTO,MOTO,AUTO
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,...,NaN,NaN,NaN,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,...,NaN,NaN,NaN,1,Point (. .),.,.,PEATON-PASAJEROS,PEATON,PASAJEROS
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,...,NaN,NaN,"LUGONES, LEOPOLDO AV.",14,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,...,NaN,NaN,NaN,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,...,NaN,NaN,NaN,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO
256,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,...,NaN,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),.,.,MOTO-PASAJEROS,MOTO,PASAJEROS
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,...,NaN,NaN,NaN,14,Point (. .),.,.,PEATON-AUTO,PEATON,AUTO
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,...,NaN,NaN,NaN,14,Point (. .),.,.,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
559,2020-0039,1,2020-09-01,2020,9,1,19:17:42,19,MURGUIONDO 2700,CALLE,...,NaN,NaN,MURGUIONDO,9,Point (. .),.,.,PEATON-CARGAS,PEATON,CARGAS


Se identifica que son las mismas filas,  estos valores faltantes se reemplazaran por los valores modales de cada comuna

In [956]:
#Creamos un nuevo df con la filas que contienen informacion en pos x y pos y
df_hechos_pro = df_hechos[(df_hechos['pos x'] != '.') & (df_hechos['pos y'] != '.')]

In [957]:
# Crea una nueva columna concatenando 'pos x' y 'pos y'
df_hechos_pro['coord_concat'] = df_hechos_pro['pos x'].astype(str) + '_' + df_hechos['pos y'].astype(str)

C:\Users\Willc\AppData\Local\Temp\ipykernel_17324\142020725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hechos_pro['coord_concat'] = df_hechos_pro['pos x'].astype(str) + '_' + df_hechos['pos y'].astype(str)


In [958]:
# Calcula la moda de la concatenación de coordenadas para cada comuna
moda_coordenadas_concat_por_comuna = df_hechos_pro.groupby('COMUNA')['coord_concat'].agg(lambda x: x.mode().iloc[0] if x.mode().any() else None)

In [959]:
# Itera sobre las modas calculadas por comuna
for comuna, moda_concat in moda_coordenadas_concat_por_comuna.items():
    # Separa la moda de la concatenación en 'pos x' y 'pos y'
    moda_x, moda_y = moda_concat.split('_')
    
    # Reemplaza los valores faltantes en 'pos x' y 'pos y' con las modas correspondientes
    mask = df_hechos['COMUNA'] == comuna
    df_hechos.loc[mask, 'pos x'] = df_hechos.loc[mask, 'pos x'].replace('.', moda_x)
    df_hechos.loc[mask, 'pos y'] = df_hechos.loc[mask, 'pos y'].replace('.', moda_y)


Verificamos los pasos aplicados

In [960]:
df_hechos[(df_hechos['pos x']=='.') & (df_hechos['pos y']=='.')]#Filtra de df por las filas coon . en las columnas poos x y pos y

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO


In [961]:
#convertimos las columnas  'pos x' y 'pos y' a tipo de dato float
df_hechos['pos x'] = df_hechos['pos x'].astype(float) 
df_hechos['pos y'] = df_hechos['pos y'].astype(float) 

In [962]:
df_hechos.rename(columns={'pos x': 'LONGITUD', 'pos y': 'LATITUD'}, inplace=True)#cambia el nombre de las columnas pos x y pos y

verificamos valores nulos

In [963]:
df_hechos.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   566
Cruce                    170
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
LONGITUD                   0
LATITUD                    0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

Se observa que las columnas 'calle', 'altura', 'cruce' y 'dirección normalizada' contienen valores nulos. La columna 'dirección normalizada'y 'Lugar del hecho' son una concatenación de 'calle' y 'cruce', lo que significa que algunos registros tienen datos solo en 'calle' o solo en 'cruce', lo que reduce la veracidad de esta información. Además, la columna 'altura' tiene una cantidad significativa de valores nulos en comparación con el total de datos.

Como resultado, para la ubicación se utilizarán las columnas 'XY (CABA)' y las coordenadas de longitud y latitud, ya que no tienen valores nulos. Debido a la falta de integridad en los datos de las columnas 'calle', 'altura', 'cruce' y 'dirección normalizada', se optará por descartar estas columnas en el análisis.

Este enfoque garantizará que la información utilizada para la ubicación sea más confiable y completa, lo que mejorará la calidad de los análisis posteriores.

In [964]:
#Se eliminan las columnas ya descartadas
df_hechos.drop(columns={'LUGAR_DEL_HECHO', 'Calle','Altura','Cruce','Dirección Normalizada'}, axis=1, inplace=True)

La columna "XY (CABA)" se refiere a la geocodificación plana, mientras que las columnas de latitud (pos y) y longitud (pos x) contienen coordenadas que representan ubicaciones geográficas precisas. Por lo tanto, podemos prescindir de la columna "XY (CABA)".

In [965]:
df_hechos.drop('XY (CABA)', axis=1, inplace=True)#Elimina la columna  XY (CABA)

In [966]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 695 entries, 0 to 695
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             695 non-null    object        
 1   N_VICTIMAS     695 non-null    int64         
 2   FECHA          695 non-null    datetime64[ns]
 3   AAAA           695 non-null    int64         
 4   MM             695 non-null    int64         
 5   DD             695 non-null    int64         
 6   HORA           695 non-null    object        
 7   HH             695 non-null    int32         
 8   TIPO_DE_CALLE  695 non-null    object        
 9   COMUNA         695 non-null    int64         
 10  LONGITUD       695 non-null    float64       
 11  LATITUD        695 non-null    float64       
 12  PARTICIPANTES  695 non-null    object        
 13  VICTIMA        695 non-null    object        
 14  ACUSADO        695 non-null    object        
dtypes: datetime64[ns](1), float6

Verificamos valores duplicados

In [967]:
df_hechos[df_hechos.duplicated()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,TIPO_DE_CALLE,COMUNA,LONGITUD,LATITUD,PARTICIPANTES,VICTIMA,ACUSADO


In [968]:
df_hechos[df_hechos['ID'].duplicated()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,TIPO_DE_CALLE,COMUNA,LONGITUD,LATITUD,PARTICIPANTES,VICTIMA,ACUSADO


In [969]:
df_hechos[df_hechos['HORA'].isnull()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,TIPO_DE_CALLE,COMUNA,LONGITUD,LATITUD,PARTICIPANTES,VICTIMA,ACUSADO


Para finalizar se exporta a un csv con los combios realizados

In [970]:
df_hechos.to_csv('../Datasets/homicidios_proc.csv', index=False)

HOJA VICTIMAS

In [971]:
#importamos las hojas del Dataset que se usaran 
df_victimas = pd.read_excel('../Datasets/homicidios.xlsx', sheet_name='VICTIMAS')

In [972]:
df_victimas.head(5)#Visualizamos las 5 primeras filas

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [973]:
df_victimas.info()#obtenemos una descripción concisa y resumida del DataFrame.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [974]:
df_victimas[df_victimas.duplicated()]#se verifican duplixados

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


In [975]:
df_victimas[df_victimas['ID_hecho'].duplicated()]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
30,2016-0041,2016-03-29,2016,3,29,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,SD,2016-03-30 00:00:00
99,2016-0126,2016-09-18,2016,9,18,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,60,SD
164,2017-0026,2017-02-26,2017,2,26,CONDUCTOR,AUTO,MASCULINO,19,2017-02-26 00:00:00
174,2017-0035,2017-03-23,2017,3,23,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,32,2017-03-23 00:00:00
175,2017-0035,2017-03-23,2017,3,23,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,30,2017-03-23 00:00:00
177,2017-0036,2017-03-29,2017,3,29,CONDUCTOR,MOTO,MASCULINO,20,2017-03-29 00:00:00
187,2017-0050,2017-04-28,2017,4,28,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,16,2017-04-28 00:00:00
238,2017-0108,2017-09-02,2017,9,2,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,10,2017-09-02 00:00:00
243,2017-0112,2017-09-10,2017,9,10,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,SD,2017-09-13 00:00:00
254,2017-0126,2017-10-14,2017,10,14,CONDUCTOR,AUTO,MASCULINO,36,2017-10-14 00:00:00


No es un error los duplicados en la columna ID_Hecho, ya que cada fila representa a una victima en cada accidente.

La columna edad debe ser un dato tipo int, por ello se verifica si la conlumna tiene SD

In [976]:
df_victimas[df_victimas['EDAD'].str.contains(r'[a-zA-Z]', na=False)].count()#Verifica si alguna fila contiene letras.

ID_hecho               53
FECHA                  53
AAAA                   53
MM                     53
DD                     53
ROL                    53
VICTIMA                53
SEXO                   53
EDAD                   53
FECHA_FALLECIMIENTO    53
dtype: int64

ya que solo tenemos 53 datos sin edad de los 717, se puede promediar la edad de las victimas por tipo de vehiculo

In [977]:
# Filtra las filas donde la edad no es 'SD'
df_victimas_proc = df_victimas[df_victimas['EDAD'] != 'SD']

In [978]:
# Convierte la columna 'EDAD' a tipo numérico para poder calcular la media
df_victimas_proc['EDAD'] = pd.to_numeric(df_victimas_proc['EDAD'], errors='coerce')

C:\Users\Willc\AppData\Local\Temp\ipykernel_17324\2498196498.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_victimas_proc['EDAD'] = pd.to_numeric(df_victimas_proc['EDAD'], errors='coerce')


In [979]:
# Calcula la media de edad para cada tipo de víctima
media_edad_victimas = df_victimas_proc.groupby('VICTIMA')['EDAD'].mean().reset_index()

In [980]:
# Itera sobre las filas de media_edad_victimas
for index, row in media_edad_victimas.iterrows():
    victima = row['VICTIMA']
    edad_media = int(row['EDAD'])
   
    # Crea una máscara para las filas donde 'VICTIMA' es igual a la víctima actual
    mask = df_victimas['VICTIMA'] == victima
    
    # Asigna la edad media a las filas que cumplen la condición
    df_victimas.loc[mask, 'EDAD'] = np.where(df_victimas.loc[mask, 'EDAD'] == 'SD', edad_media, df_victimas.loc[mask, 'EDAD'])

In [981]:
df_victimas['EDAD'] = df_victimas['EDAD'].astype(int)#se cambia el tipo de dato a int

Verificamos la columna FECHA_FALLECIMIENTO

In [982]:
df_victimas[df_victimas['FECHA_FALLECIMIENTO']== 'SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
16,2016-0022,2016-02-21,2016,2,21,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,41,SD
19,2016-0027,2016-02-28,2016,2,28,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,34,SD
22,2016-0031,2016-03-08,2016,3,8,CONDUCTOR,MOTO,MASCULINO,21,SD
33,2016-0045,2016-04-11,2016,4,11,CONDUCTOR,MOTO,MASCULINO,31,SD
...,...,...,...,...,...,...,...,...,...,...
221,2017-0089,2017-07-13,2017,7,13,SD,SD,MASCULINO,23,SD
242,2017-0112,2017-09-10,2017,9,10,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,1,SD
246,2017-0115,2017-09-19,2017,9,19,CONDUCTOR,MOTO,MASCULINO,34,SD
253,2017-0126,2017-10-14,2017,10,14,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,39,SD


In [983]:
df_victimas[df_victimas['FECHA_FALLECIMIENTO']==df_victimas['FECHA']]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
5,2016-0008,2016-01-24,2016,1,24,CONDUCTOR,MOTO,MASCULINO,30,2016-01-24 00:00:00
7,2016-0010,2016-01-29,2016,1,29,CONDUCTOR,MOTO,MASCULINO,18,2016-01-29 00:00:00
...,...,...,...,...,...,...,...,...,...,...
706,2021-0086,2021-11-25,2021,11,25,PEATON,PEATON,MASCULINO,75,2021-11-25 00:00:00
707,2021-0087,2021-11-27,2021,11,27,PEATON,PEATON,FEMENINO,73,2021-11-27 00:00:00
711,2021-0091,2021-12-11,2021,12,11,CONDUCTOR,MOTO,MASCULINO,24,2021-12-11 00:00:00
712,2021-0092,2021-12-12,2021,12,12,PEATON,PEATON,FEMENINO,50,2021-12-12 00:00:00


Se identifica que en la mayoria de los casos las victimas fallecen el mismos dia, para estos 68 casos sin datos tomaremos la fecha de cuando ocurrio el accidente

In [984]:
# Reemplazar 'SD' por NaN en la columna 'FECHA_FALLECIMIENTO'
df_victimas['FECHA_FALLECIMIENTO'] = df_victimas['FECHA_FALLECIMIENTO'].replace('SD', np.nan)

In [985]:
# Llenar los valores NaN en 'FECHA_FALLECIMIENTO' con los valores de 'fecha'
df_victimas['FECHA_FALLECIMIENTO'].fillna(df_victimas['FECHA'], inplace=True)

In [986]:
#Cambiar el tipo de dato de la columna FECHA_FALLECIMIENTO
df_victimas['FECHA_FALLECIMIENTO'] = pd.to_datetime(df_victimas['FECHA_FALLECIMIENTO'])


In [987]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    int32         
 9   FECHA_FALLECIMIENTO  717 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int32(1), int64(3), object(4)
memory usage: 53.3+ KB


Para reemplazar los datos faltantes de las victimas tomaramos la informacion de la df_hechos para estos casos

In [988]:
# Identificar las filas con valores faltantes en df_victimas
filas_faltantes = df_victimas[df_victimas['VICTIMA'] == 'SD']

In [989]:
# Convertir el índice de df_victimas en una columna
filas_faltantes.reset_index(inplace=True)

In [990]:
# Completar los valores faltantes usando merge
df_victimas_completado = filas_faltantes.merge(df_hechos[['ID', 'VICTIMA']], left_on='ID_hecho', right_on='ID', how='left')

In [991]:
# Eliminar la columna redundante 'ID' resultante del merge
df_victimas_completado.drop(columns={'ID', 'VICTIMA_x'}, axis=1, inplace=True)

In [992]:
# Renombrar la columna 'VICTIMA_y' a 'VICTIMA'
df_victimas_completado.rename(columns={'VICTIMA_y': 'VICTIMA'}, inplace=True)


In [993]:
# Convierte la columna index en el indicice del df
df_victimas_completado.set_index('index', inplace=True)

In [994]:
# Actualizar df_victimas con los valores completados
df_victimas.update(df_victimas_completado)

In [995]:
df_victimas[df_victimas['VICTIMA'] == 'SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
141,2016-0174,2016-12-27,2016,12,27,SD,SD,SD,54,2016-12-27


El ultimo dato corresponde a la fila que fue eliminada en el df_hechos, por ende tambien se elimina en el df_victimas

In [996]:
# Eliminar la fila con índice 141
df_victimas.drop(141, axis=0, inplace=True)

Ahore revisemos la columna ROL

In [997]:
# Identificar las filas con valores faltantes en df_victimas columna ROL
df_victimas[df_victimas['ROL']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
36,2016-0049,2016-04-17,2016,4,17,SD,MOTO,SD,54,2016-04-17
39,2016-0052,2016-04-20,2016,4,20,SD,MOTO,SD,31,2016-04-20
63,2016-0085,2016-06-29,2016,6,29,SD,MOTO,MASCULINO,31,2016-06-29
77,2016-0101,2016-08-07,2016,8,7,SD,MOTO,MASCULINO,67,2016-08-07
89,2016-0115,2016-09-02,2016,9,2,SD,MOTO,MASCULINO,54,2016-09-02
167,2017-0029,2017-03-07,2017,3,7,SD,MOTO,MASCULINO,34,2017-03-07
208,2017-0074,2017-06-04,2017,6,4,SD,MOTO,MASCULINO,70,2017-06-04
221,2017-0089,2017-07-13,2017,7,13,SD,MOTO,MASCULINO,23,2017-07-13
280,2017-0155,2017-12-12,2017,12,12,SD,PEATON,MASCULINO,77,2017-12-12
390,2018-0100,2018-09-20,2018,9,20,SD,MOTO,FEMENINO,33,2018-09-20


Para estos caso tomaremos la moda de la victima por ROL para completar los datos del ROL

In [998]:
# Cargamo un nuevo df con los datos not null
df_v_rol = df_victimas[df_victimas['ROL']!='SD']

In [999]:
# Calcular la moda de 'victima' para cada 'sexo'
moda_rol = df_v_rol.groupby('VICTIMA')['ROL'].agg(lambda x: x.mode().iloc[0])

In [1000]:
# Reemplazar los valores "SD" en 'ROL' con la moda correspondiente de 'victima'
for index, row in df_victimas.iterrows():
    if row['ROL'] == 'SD' and row['VICTIMA'] in moda_rol.index:
        df_victimas.at[index, 'ROL'] = moda_rol[row['VICTIMA']]


In [1001]:
# Identificar las filas con valores faltantes en df_victimas columna ROL
df_victimas[df_victimas['ROL']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


Analizamos la columna SEXO

In [1002]:
df_victimas['SEXO'].value_counts()

SEXO
MASCULINO    545
FEMENINO     166
SD             5
Name: count, dtype: int64

Se reemplazaran los datos faltantes de acuerdo a las modas segun la victima

In [1003]:
# Filtra el df por los datos not null
df_victimas_filtro = df_victimas[df_victimas['SEXO']!= 'SD']

In [1004]:
# Calcular la moda de 'victima' para cada 'sexo'
moda_por_sexo = df_victimas_filtro.groupby('VICTIMA')['SEXO'].agg(lambda x: x.mode().iloc[0])

In [1005]:
# Reemplazar los valores "SD" en 'sexo' con la moda correspondiente de 'victima'
for index, row in df_victimas.iterrows():
    if row['SEXO'] == 'SD' and row['VICTIMA'] in moda_por_sexo.index:
        df_victimas.at[index, 'SEXO'] = moda_por_sexo[row['VICTIMA']]

In [922]:
df_victimas[df_victimas['SEXO']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


Para finalizar se exporta como csv

In [1008]:
df_victimas.to_csv('../Datasets/homicidios_victimas_proc.csv', index=False)

Se crea una tabla con la población estimada en CABA para los años del 2016 al 2021

In [ ]:
# Definir los datos iniciales
poblacion_inicial = 2890151
poblacion_final = 3121707
tasa_crecimiento_anual = (poblacion_final / poblacion_inicial) ** (1 / (2022 - 2010)) - 1

# Crear una lista para almacenar los datos de la tabla
data = []

# Calcular la población estimada por semestre y agregar los datos a la lista
for year in range(2010, 2023):
    for semester in range(1, 3):  # 1 para primer semestre, 2 para segundo semestre
        poblacion_estimada = round(poblacion_inicial * (1 + tasa_crecimiento_anual) ** ((year - 2010) * 2 + semester - 1))
        data.append([f'{year}-S{semester}', poblacion_estimada])

# Crear el DataFrame a partir de la lista de datos
df = pd.DataFrame(data, columns=['Semestre', 'Población Estimada'])

# Mostrar el DataFrame
print(df)

   Semestre  Población Estimada
0   2010-S1             2890151
1   2010-S2             2908773
2   2011-S1             2927515
3   2011-S2             2946378
4   2012-S1             2965362
5   2012-S2             2984469
6   2013-S1             3003698
7   2013-S2             3023052
8   2014-S1             3042530
9   2014-S2             3062134
10  2015-S1             3081864
11  2015-S2             3101722
12  2016-S1             3121707
13  2016-S2             3141821
14  2017-S1             3162065
15  2017-S2             3182439
16  2018-S1             3202944
17  2018-S2             3223581
18  2019-S1             3244352
19  2019-S2             3265256
20  2020-S1             3286295
21  2020-S2             3307469
22  2021-S1             3328780
23  2021-S2             3350229
24  2022-S1             3371815
25  2022-S2             3393541


In [ ]:
# Seleccionar los semestres de los años 2016 a 2021
subset_df = df.loc[df['Semestre'].str[:4].astype(int).between(2016, 2021)]


In [ ]:
subset_df

,Semestre,Población Estimada
12,2016-S1,3121707
13,2016-S2,3141821
14,2017-S1,3162065
15,2017-S2,3182439
16,2018-S1,3202944
17,2018-S2,3223581
18,2019-S1,3244352
19,2019-S2,3265256
20,2020-S1,3286295
21,2020-S2,3307469
